In [1]:
import numpy as np
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
import os 
from PIL import Image
import matplotlib.pyplot as plt


In [2]:
pi = pd.read_excel('LungData/LUAD Clinical Data.xlsx')


celltype = loadmat('LungData/LUAD_IMC_CellType/LUAD_D001.mat')  # load mat-file
# celltype = loadmat('LungData/LUAD_IMC_MaskTif/LUAD_D001.mat')  # load mat-file
segm_ari = loadmat('LungData/LUAD_IMC_Segmentation/LUAD_D001/allRegionIndices.mat')  # load mat-file
segm_nms = loadmat('LungData/LUAD_IMC_Segmentation/LUAD_D001/nuclei_multiscale.mat')  # load mat-file


In [3]:
segm_nms['nucleiImage'].shape

(993, 1007)

In [4]:
for i in celltype:
    print(i)

__header__
__version__
__globals__
Boundaries
cellTypes
allLabels
allCols


In [5]:
def index_to_coord(vec, height):
    y = vec//height
    x = vec%height
    return x,y

def bound_to_2d(bound_vec, width, height):
    b = np.zeros(height*width, dtype=np.int8)
    b[bound_vec] = 1
    b = np.reshape(b,[width,height]).T
    return b

def avg_idx(xs, ys):
    return round(np.mean(xs)), round(np.mean(ys))

def emp_to_nan(l):
    res = np.array([])
    for i in range(len(l)):
        try:
            res = np.append(res, l[i][0][0])
        except:
            res = np.append(res, 'NA')
    return res


In [23]:
# l = emp_to_nan(celltype['cellTypes'])
# l[:20]

In [24]:
# i = bound_to_2d(temp, 1007, 993)
# plt.imshow(i)
# plt.show()

In [25]:
# plt.imshow(segm_nms['nucleiImage'])
# plt.show()

# plt.imshow(segm_nms['nucleiOccupancyIndexed'])
# plt.show()
# plt.imshow(segm_nms['occupancy_image'])

# plt.show()
# # im.show(segm_nms['nucleiImage'])

In [6]:
# print(celltype['Boundaries'].shape)
# pnum


master_dict = {}
for i in range(1, 417):
    no = format(i, '03d')
    pat = 'LUAD_D' + no
    path_celltype = 'LungData/LUAD_IMC_CellType/' + pat + '.mat'
    path_seg_nms = 'LungData/LUAD_IMC_Segmentation/'+ pat +'/nuclei_multiscale.mat'
    celltype = loadmat(path_celltype)  # load mat-file
    segm_nms = loadmat(path_seg_nms)  # load mat-file

# pat = 'LUAD_D001'
    patientdict = {}
# patientdict
    height, width = segm_nms['nucleiImage'].shape
    # celltype_list[#][0] # is cellno
    celltype_list = emp_to_nan(celltype['cellTypes'])

    for cellno, cell in enumerate(segm_nms['Boundaries'][0]):
        xcell,ycell = index_to_coord(cell, height)
        x,y = avg_idx(xcell, ycell)
        c_type = celltype_list[cellno]
        patientdict[cellno] = np.array([x,y,c_type])
        
    master_dict[pat] = patientdict

master_dict['LUAD_D001']

{0: array(['93', '159', 'Cancer'], dtype='<U11'),
 1: array(['74', '186', 'Tc'], dtype='<U11'),
 2: array(['167', '131', 'Tc'], dtype='<U11'),
 3: array(['134', '116', 'Cancer'], dtype='<U11'),
 4: array(['142', '122', 'Endothelial cell'], dtype='<U16'),
 5: array(['133', '152', 'Cl MAC'], dtype='<U11'),
 6: array(['153', '101', 'B cell'], dtype='<U11'),
 7: array(['91', '179', 'Th'], dtype='<U11'),
 8: array(['141', '197', 'Cancer'], dtype='<U11'),
 9: array(['144', '168', 'NA'], dtype='<U11'),
 10: array(['120', '149', 'Th'], dtype='<U11'),
 11: array(['170', '107', 'Alt MAC'], dtype='<U11'),
 12: array(['197', '140', 'Cancer'], dtype='<U11'),
 13: array(['198', '76', 'Cancer'], dtype='<U11'),
 14: array(['130', '164', 'Cancer'], dtype='<U11'),
 15: array(['114', '173', 'Cancer'], dtype='<U11'),
 16: array(['178', '87', 'Cancer'], dtype='<U11'),
 17: array(['115', '164', 'Cl MAC'], dtype='<U11'),
 18: array(['196', '69', 'Cancer'], dtype='<U11'),
 19: array(['181', '97', 'Treg'], dty

In [7]:
master_dict['LUAD_D415']

{0: array(['74', '105', 'Cancer'], dtype='<U11'),
 1: array(['35', '87', 'Cancer'], dtype='<U11'),
 2: array(['64', '35', 'Cancer'], dtype='<U11'),
 3: array(['6', '176', 'Cancer'], dtype='<U11'),
 4: array(['76', '57', 'Cancer'], dtype='<U11'),
 5: array(['121', '72', 'Endothelial cell'], dtype='<U16'),
 6: array(['135', '45', 'Endothelial cell'], dtype='<U16'),
 7: array(['4', '130', 'Endothelial cell'], dtype='<U16'),
 8: array(['79', '80', 'NA'], dtype='<U11'),
 9: array(['127', '135', 'Cancer'], dtype='<U11'),
 10: array(['153', '55', 'B cell'], dtype='<U11'),
 11: array(['117', '30', 'Alt MAC'], dtype='<U11'),
 12: array(['143', '7', 'Cl MAC'], dtype='<U11'),
 13: array(['167', '94', 'Cancer'], dtype='<U11'),
 14: array(['190', '160', 'Cancer'], dtype='<U11'),
 15: array(['27', '139', 'Endothelial cell'], dtype='<U16'),
 16: array(['135', '59', 'Endothelial cell'], dtype='<U16'),
 17: array(['11', '160', 'B cell'], dtype='<U11'),
 18: array(['126', '48', 'Endothelial cell'], dtyp

In [8]:
pi = pd.read_excel('LungData/LUAD Clinical Data.xlsx')
patientinfocols = pi.columns
df = pd.DataFrame(master_dict['LUAD_D001']).T
df.columns = ['x_pix', 'y_pix', 'celltype']

df = pd.concat([df, pd.concat([pi[pi['Key'] == 'LUAD_D001']]*len(df), ignore_index=True)], axis=1)
df['ID'] = df['Key'].str[5:] + '_' + df.index.astype(str)

df

,x_pix,y_pix,celltype,Key,"Sex (Male: 0, Female: 1)","Age (<75: 0, ≥75: 1)","BMI (<30: 0, ≥30: 1)","Smoking Status (Smoker: 0, Non-smoker:1)","Pack Years (1-30: 0, ≥30: 1)","Stage (I-II: 0, III-IV:1)","Progression (No: 0, Yes: 1)","Death (No: 0, Yes: 1)",Survival or loss to follow-up (years),"Predominant histological pattern (Lepidic:1, Papillary: 2, Acinar: 3, Micropapillary: 4, Solid: 5)",ID
0,93,159,Cancer,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_0
1,74,186,Tc,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_1
2,167,131,Tc,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_2
3,134,116,Cancer,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_3
4,142,122,Endothelial cell,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4512,232,953,Th,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4512
4513,241,953,Cl MAC,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4513
4514,359,951,Endothelial cell,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4514
4515,332,975,Endothelial cell,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4515


In [9]:
# stacking patient dfs 
# index is cell no per patient
# 25 mum distacne neightbour

for i in range(2, 417):
    no = format(i, '03d')
    pat = 'LUAD_D' + no
    df2 = pd.DataFrame(master_dict[pat]).T
    df2.columns = ['x_pix', 'y_pix', 'celltype']

    df2 = pd.concat([df2, pd.concat([pi[pi['Key'] == pat]]*len(df2), ignore_index=True)], axis=1)
    df2['ID'] = df2['Key'].str[5:] + '_' + df2.index.astype(str)
    df = pd.concat([df,df2], axis=0)
df

,x_pix,y_pix,celltype,Key,"Sex (Male: 0, Female: 1)","Age (<75: 0, ≥75: 1)","BMI (<30: 0, ≥30: 1)","Smoking Status (Smoker: 0, Non-smoker:1)","Pack Years (1-30: 0, ≥30: 1)","Stage (I-II: 0, III-IV:1)","Progression (No: 0, Yes: 1)","Death (No: 0, Yes: 1)",Survival or loss to follow-up (years),"Predominant histological pattern (Lepidic:1, Papillary: 2, Acinar: 3, Micropapillary: 4, Solid: 5)",ID
0,93,159,Cancer,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_0
1,74,186,Tc,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_1
2,167,131,Tc,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_2
3,134,116,Cancer,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_3
4,142,122,Endothelial cell,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2881,281,879,Cl MAC,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2881
2882,385,907,NA,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2882
2883,363,992,Cancer,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2883
2884,357,994,Cancer,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2884


In [38]:
df

,x_pix,y_pix,celltype,Key,"Sex (Male: 0, Female: 1)","Age (<75: 0, ≥75: 1)","BMI (<30: 0, ≥30: 1)","Smoking Status (Smoker: 0, Non-smoker:1)","Pack Years (1-30: 0, ≥30: 1)","Stage (I-II: 0, III-IV:1)","Progression (No: 0, Yes: 1)","Death (No: 0, Yes: 1)",Survival or loss to follow-up (years),"Predominant histological pattern (Lepidic:1, Papillary: 2, Acinar: 3, Micropapillary: 4, Solid: 5)",ID
0,93,159,Cancer,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_0
1,74,186,Tc,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_1
2,167,131,Tc,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_2
3,134,116,Cancer,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_3
4,142,122,Endothelial cell,LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3,D001_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2881,281,879,Cl MAC,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2881
2882,385,907,NA,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2882
2883,363,992,Cancer,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2883
2884,357,994,Cancer,LUAD_D416,1,0,1,0.0,1.0,0.0,0.0,0,9.386,3,D416_2884


In [10]:
current_directory = os.getcwd()

output_path = f"{current_directory}/celldata/"
os.makedirs(output_path, exist_ok=True)

df.to_csv(f"{output_path}20240208_cell+patient_info.csv", index = False)

In [11]:
# Format the data for celldistance py script
# celldata = celldata[['ROI_name', 'ObjectNumber', 'study', 'class','Location_Center_X', 'Location_Center_Y']]
celldata = df[['ID', 'celltype','Key', 'x_pix', 'y_pix']]
celldata = celldata.rename(columns={"Key":"ROI_name","ID": "cellID", "celltype":"class", 'x_pix':'Location_Center_X', 'y_pix':'Location_Center_Y'})
# celldata['ROI_name'] = 1
celldata['study'] = 1

celldata.to_csv(f"{output_path}20240208_celldata.csv", index = False)

celldata

,cellID,class,ROI_name,Location_Center_X,Location_Center_Y,study
0,D001_0,Cancer,LUAD_D001,93,159,1
1,D001_1,Tc,LUAD_D001,74,186,1
2,D001_2,Tc,LUAD_D001,167,131,1
3,D001_3,Cancer,LUAD_D001,134,116,1
4,D001_4,Endothelial cell,LUAD_D001,142,122,1
...,...,...,...,...,...,...
2881,D416_2881,Cl MAC,LUAD_D416,281,879,1
2882,D416_2882,NA,LUAD_D416,385,907,1
2883,D416_2883,Cancer,LUAD_D416,363,992,1
2884,D416_2884,Cancer,LUAD_D416,357,994,1


In [40]:
celldata

,cellID,class,ROI_name,Location_Center_X,Location_Center_Y,study
0,D001_0,Cancer,LUAD_D001,93,159,1
1,D001_1,Tc,LUAD_D001,74,186,1
2,D001_2,Tc,LUAD_D001,167,131,1
3,D001_3,Cancer,LUAD_D001,134,116,1
4,D001_4,Endothelial cell,LUAD_D001,142,122,1
...,...,...,...,...,...,...
2881,D416_2881,Cl MAC,LUAD_D416,281,879,1
2882,D416_2882,NA,LUAD_D416,385,907,1
2883,D416_2883,Cancer,LUAD_D416,363,992,1
2884,D416_2884,Cancer,LUAD_D416,357,994,1
